# Put simulation data in a usable form

In [1]:
import pylab as p
import numpy as np
from astropy.io import fits
import glob

# For saving images
import scipy.misc

/Library/Python/2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
data_dir = "../data/lensing_large/"
out_dir = "../data/lensing_small/"

In [3]:
# Takes a fits filename (including filepath), and returns a set of cut up images
def get_images(file_name, nx_pixels=256, ny_pixels=256):
    
    # Load first file
    snap1 = fits.open(file_name)
    scidata = snap1[0].data

    # Number of images that will fit in each image
    nx_images = len(scidata) / nx_pixels
    ny_images = len(scidata[0]) / ny_pixels

    # Total number of images
    n_images = nx_images * ny_images

    # Create numpy array to hold images
    images = np.empty((n_images, nx_pixels, ny_pixels))


    for i in np.arange(nx_images):
        for j in np.arange(ny_images):
            index_image = i*nx_images + j
            images[index_image] = scidata[nx_pixels*i:nx_pixels*(i+1), ny_pixels*j:ny_pixels*(j+1)]
        
    return images

In [34]:
def degrade_image(images, bin_by):

    n_images = len(images[::])

    nx_pixels_in = len(images[0][0])
    ny_pixels_in = len(images[0].T[0])

    nx_pixels_out = nx_pixels_in / bin_by
    ny_pixels_out = ny_pixels_in / bin_by

    pictures = np.empty((n_images, nx_pixels_out, ny_pixels_out))

    for p in np.arange(n_images):

        vecs = np.zeros((nx_pixels_in, ny_pixels_out))
        for i in np.arange(nx_pixels_in):
            vecs[i] = 0.5 * (images[p][i][::2] + images[p][i][1::2])

        pictures[p] = 0.5 * (vecs[::2] + vecs[1::2])

    return pictures

In [35]:
file_names = glob.glob(data_dir + "*.fits")

i = 0
for f in file_names:
    images = get_images(f)
    pic_out = degrade_image(images, 2)

    print pic_out.shape

    for j in np.arange(len(images[::])):
        i += 1
        scipy.misc.imsave(out_dir + 'outfile_' + str(i) + '.jpg', images[j])
        

(64, 128, 128)
(64, 128, 128)
